В данном ноутбуке рассмотрим экстрактивный подход к суммаризации текста

In [1]:
import pandas as pd


news = pd.read_csv('../data/news10000.csv')

In [2]:
new1 = news.iloc[0]
text = new1['text']
sentences = text.split('.')
len(sentences)

12

In [3]:
sentences_for_word2vec = news['text'].apply(lambda x: x.split('.')) # split text into sentences

In [5]:
import pymorphy2


morph = pymorphy2.MorphAnalyzer()
lemmatized_sentences = []

for sentence in sentences:
    words = sentence.split()
    lemmatized_words = [morph.parse(word)[0].normal_form for word in words]
    lemmatized_sentence = ' '.join(lemmatized_words)
    lemmatized_sentences.append(lemmatized_sentence)

print(lemmatized_sentences)


AttributeError: module 'inspect' has no attribute 'getargspec'

In [ ]:
lemmatized_sentences_word2vec = []

for i in sentences_for_word2vec:
    for sentence in i:
        words = sentence.split()
        lemmatized_words = [morph.parse(word)[0].normal_form for word in words]
        lemmatized_sentence = ' '.join(lemmatized_words)
        lemmatized_sentences_word2vec.append(lemmatized_sentence)

In [ ]:
lemmatized_sentences_word2vec = [sentence.split() for sentence in lemmatized_sentences_word2vec]

Экстрактивная суммаризация на основе вхождения общих слов

In [ ]:
def calculate_similarity(sentences):
    similarities = []
    
    for i in range(len(sentences)):
        for j in range(i+1, len(sentences)):
            sentence1 = sentences[i]
            sentence2 = sentences[j]
            
            words1 = sentence1.split()
            words2 = sentence2.split()
            
            common_words = set(words1) & set(words2)
            similarity = len(common_words) / (len(words1) + len(words2))
            if similarity > 0:
                similarities.append((sentence1, sentence2, similarity, i , j))
    
    return similarities


In [ ]:
def ranging(similarities, ln):
    result = [(0, i) for i in range(ln)]
    for i in similarities:
        if i[2] > 0:
            result[i[3]][0] += 1
            result[i[4]][0] += 1
    return result

In [ ]:
from pprint import pprint


ln_sent = len(lemmatized_sentences)
similarities = calculate_similarity(lemmatized_sentences)
rang = ranging(similarities, ln_sent)
rang.sort(key=lambda x: x[0], reverse=True)
pprint(range[:2])

[('неприятель, приблизиться с север к осовца начать артиллерийский борьба с '
  'крепость',
  'попытка германский пехота пробиться близкий к крепость отразить',
  0.10526315789473684,
  1,
  4),
 ('в артиллерийский бой принимать участие тяжёлый калибр',
  'в галиция мы занять дембица',
  0.08333333333333333,
  2,
  5)]
 Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью
 Попытка германской пехоты пробиться ближе к крепости отражена
 В артиллерийском бою принимают участие тяжелые калибры
 В Галиции мы заняли Дембицу


Экстрактивная суммаризация на основе обученных векторных представлений

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(lemmatized_sentences_word2vec, min_count=1)

model.train(lemmatized_sentences_word2vec, total_examples=model.corpus_count, epochs=10)

(13229930, 15177920)

In [ ]:
sentence_embedding = []

for sentence in lemmatized_sentences:
    words = sentence.split()
    embedding = sum([model.wv[word] for word in words])
    sentence_embedding.append(embedding)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = []

for i in range(len(sentence_embedding)):
    for j in range(i+1, len(sentence_embedding)):
        embedding1 = sentence_embedding[i]
        embedding2 = sentence_embedding[j]
        if isinstance(embedding1, int) or isinstance(embedding2, int):
            continue

        if embedding1.all() == 0 or embedding2.all() == 0:
            continue

        similarity = cosine_similarity([embedding1], [embedding2])[0][0]
        similarity_matrix.append((similarity, i, j))

similarity_matrix


[(0.31649372, 0, 1),
 (0.4989151, 0, 2),
 (0.37511656, 0, 3),
 (0.37333974, 0, 4),
 (0.3338399, 0, 5),
 (0.46066606, 0, 6),
 (0.37298033, 0, 7),
 (0.42174768, 0, 8),
 (0.55801576, 0, 9),
 (0.14970517, 0, 10),
 (0.37316564, 1, 2),
 (0.59505117, 1, 3),
 (0.6071144, 1, 4),
 (0.33905736, 1, 5),
 (0.72942156, 1, 6),
 (0.2393505, 1, 7),
 (0.44585058, 1, 8),
 (0.48829982, 1, 9),
 (0.20476171, 1, 10),
 (0.40791482, 2, 3),
 (0.2869617, 2, 4),
 (0.26325667, 2, 5),
 (0.48787487, 2, 6),
 (0.40069795, 2, 7),
 (0.4622263, 2, 8),
 (0.45909458, 2, 9),
 (0.21284302, 2, 10),
 (0.37572986, 3, 4),
 (0.28212357, 3, 5),
 (0.6137512, 3, 6),
 (0.20154104, 3, 7),
 (0.39688435, 3, 8),
 (0.4170017, 3, 9),
 (0.60179204, 3, 10),
 (0.43817982, 4, 5),
 (0.55983496, 4, 6),
 (0.26581037, 4, 7),
 (0.38130057, 4, 8),
 (0.47588485, 4, 9),
 (0.18579806, 4, 10),
 (0.40776572, 5, 6),
 (0.38305175, 5, 7),
 (0.41879508, 5, 8),
 (0.7073965, 5, 9),
 (0.095969535, 5, 10),
 (0.38034979, 6, 7),
 (0.6048609, 6, 8),
 (0.61098987, 6,

In [ ]:
ranked_variables = sorted(similarity_matrix, key=lambda x: x[0], reverse=True)


In [ ]:
result = ranked_variables[:2]

for i in range(2):
    print(sentences[result[i][1]])
    print(sentences[result[i][2]])

 Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью
 Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили
 В Галиции мы заняли Дембицу
 На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии
